In [ ]:
# -*- coding: utf-8 -*-
# Copyright (c) 2017-present, Facebook, Inc.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
#

"""
Created on Sat Sep 19 20:55:56 2015

@author: liangshiyu
"""

from __future__ import print_function
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
from scipy import misc
import random

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [ ]:
def odin(model=None, id_testloader=None, out_testloader=None, magnitude=0.0014, temperature=1000, std=(0,0,0), file_path=''):
    g1 = open(f"{file_path}/confidence_Our_In.txt", 'w')
    g2 = open(f"{file_path}/confidence_Our_Out.txt", 'w')
    
    print("Processing in-distribution images")
    
    criterion = nn.CrossEntropyLoss()
    
########################################In-distribution###########################################
    for j, data in enumerate(id_testloader):
        images, _ = data
        batch_size = images.size(0)
        
        inputs = images.to(device).requires_grad_()
        outputs = model(inputs)
        
        # Calculating the confidence of the output, no perturbation added here, no temperature scaling used
        nnOutputs = outputs.detach().cpu().numpy()
        nnOutputs = np.exp(nnOutputs - np.max(nnOutputs, axis=1, keepdims=True))
        nnOutputs = nnOutputs / np.sum(nnOutputs, axis=1, keepdims=True)
        
        # Using temperature scaling
        outputs = outputs / temperature
	
        # Calculating the perturbation we need to add, that is,
        # the sign of gradient of cross entropy loss w.r.t. input
        maxIndexTemp = torch.argmax(outputs, dim=1)
        labels = maxIndexTemp.to(device)
        
        loss = criterion(outputs, labels)
        loss.backward()
        
        # Normalizing the gradient to binary in {0, 1}
        gradient = torch.ge(inputs.grad.data, 0)
        gradient = (gradient.float() - 0.5) * 2
        
        # Normalizing the gradient to the same space of image
        gradient[0][0] = (gradient[0][0]) / std[0]
        gradient[0][1] = (gradient[0][1]) / std[1]
        gradient[0][2] = (gradient[0][2]) / std[2]
        
        # Adding small perturbations to images
        tempInputs = torch.add(inputs.detach(), gradient, alpha=-magnitude)
        outputs = model(tempInputs)
        outputs = outputs / temperature
        
        # Calculating the confidence after adding perturbations
        nnOutputs = outputs.data.cpu().numpy()
        nnOutputs = np.exp(nnOutputs - np.max(nnOutputs, axis=1, keepdims=True))
        nnOutputs = nnOutputs / np.sum(nnOutputs, axis=1, keepdims=True)
        
        for i in range(batch_size):
            g1.write("{}, {}, {}\n".format(temperature, magnitude, np.max(nnOutputs[i])))
            
        images.grad = None
        
    print("Processing out-of-distribution images")
###################################Out-of-Distributions#####################################
    for j, data in enumerate(out_testloader):
        images, _ = data
        batch_size = images.size(0)
    
        inputs = images.to(device).requires_grad_()
        outputs = model(inputs)
        
        # Calculating the confidence of the output, no perturbation added here
        nnOutputs = outputs.detach().cpu().numpy()
        nnOutputs = np.exp(nnOutputs - np.max(nnOutputs, axis=1, keepdims=True))
        nnOutputs = nnOutputs / np.sum(nnOutputs, axis=1, keepdims=True)
        
        # Using temperature scaling
        outputs = outputs / temperature  
  
        # Calculating the perturbation we need to add, that is,
        # the sign of gradient of cross entropy loss w.r.t. input
        maxIndexTemp = torch.argmax(outputs, dim=1)
        labels = maxIndexTemp.to(device)
        
        loss = criterion(outputs, labels)
        loss.backward()
        
        # Normalizing the gradient to binary in {0, 1}
        gradient = torch.ge(inputs.grad.data, 0)
        gradient = (gradient.float() - 0.5) * 2
        
        # Normalizing the gradient to the same space of image
        gradient[0][0] = (gradient[0][0]) / std[0]
        gradient[0][1] = (gradient[0][1]) / std[1]
        gradient[0][2] = (gradient[0][2]) / std[2]
        
        # Adding small perturbations to images
        tempInputs = torch.add(inputs.detach(), gradient, alpha=-magnitude)
        outputs = model(tempInputs)
        outputs = outputs / temperature
        
        # Calculating the confidence after adding perturbations
        nnOutputs = outputs.data.cpu().numpy()
        nnOutputs = np.exp(nnOutputs - np.max(nnOutputs, axis=1, keepdims=True))
        nnOutputs = nnOutputs / np.sum(nnOutputs, axis=1, keepdims=True)
        
        for i in range(batch_size):
            g2.write("{}, {}, {}\n".format(temperature, magnitude, np.max(nnOutputs[i])))
            
        images.grad = None
        
    g1.close()
    g2.close()        